# Setup

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import os
project_folder = '/gdrive/MyDrive/ProjectCIRI'
my_module_path = os.path.join(project_folder, 'code', 'ciri_utils')

In [3]:
!pip install $my_module_path

Processing /gdrive/MyDrive/ProjectCIRI/code/ciri_utils
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 9.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-ma

In [4]:
import ray

from ray import tune
from ciri_utils.engine_v2 import CIRI_trainer

In [5]:
# Paths for the dataset
root_base = os.path.join(project_folder, 'Incidents-subset')
root_augmented = os.path.join(project_folder, 'augmented_images')

data_folders=[root_base, root_augmented]
persistence_path = os.path.join(project_folder, 'checkpoints', 'HPT_wide_resnet_50')

# Preparation

In [6]:
selected_model="wide_resnet50_2"

In [ ]:
ciri_trainer = CIRI_trainer(model=selected_model,
                            data_folders=data_folders,
                            data_prop=0.8,
                            sample_indices=0.2)

# Hyperparameter tuning

Perform hyper parameter tuning with nested cross-validation on a 20% sample of the dataset.

In [7]:
search_space={
	'epochs': tune.choice([5, 10, 20]),
	'batch_size': tune.choice([32, 64]),
	'lr': tune.loguniform(1e-4, 1e-2)
}

In [ ]:
hp_nest_cv = ciri_trainer.cross_validate(
    run_name="wide_resnet50_hpt",
    config=search_space,
    outer_cv_k=5,
    inner_cv_k=3,
    tune_hyperparams=True,
    num_samples=5,
    results_persist_dir=persistence_path,
    start_fold=(4, 0)
)

Outer fold 4, inner fold 0 - number of samples: 1281
Tuning hyperparameters for wide_resnet50_hpt_outer_4_inner_0...
Defaulting to ASHA scheduler (no scheduler provided or not an instance of TrialScheduler)


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-04-14 07:52:13,556	INFO worker.py:1752 -- Started a local Ray instance.
2024-04-14 07:52:14,946	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2024-04-14 07:52:14,963	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+----------------------------------------------------------------------+
| Configuration for experiment     wide_resnet50_hpt_outer_4_inner_0   |
+----------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator               |
| Scheduler                        AsyncHyperBandScheduler             |
| Number of trials                 5                                   |
+----------------------------------------------------------------------+

View detailed results here: /root/ray_results/wide_resnet50_hpt_outer_4_inner_0
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-14_07-52-09_892030_20145/artifacts/2024-04-14_07-52-14/wide_resnet50_hpt_outer_4_inner_0/driver_artifacts`

Trial status: 5 PENDING
Current time: 2024-04-14 07:52:15. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:V100)
+----------------------------------------------

(TrainTrainable pid=20570) 2024-04-14 07:52:21.506807: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=20570) 2024-04-14 07:52:21.506873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=20570) 2024-04-14 07:52:21.508720: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainTrainable pid=20570) 2024-04-14 07:52:23.440172: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Trial TorchTrainer_e8bbe_00000 started with configuration:
+------------------------------------------------------------------+
| Trial TorchTrainer_e8bbe_00000 config                            |
+------------------------------------------------------------------+
| train_loop_config/batch_size                                  64 |
| train_loop_config/epochs                                      10 |
| train_loop_config/lr                      0.00030405490309504623 |
| train_loop_config/train_test_idx            ...20, 11731, 1157]) |
+------------------------------------------------------------------+


...


Trial TorchTrainer_a1400_00004 completed after 1 iterations at 2024-04-14 09:01:48. Total running time: 35min 45s
+---------------------------------------------------------------+
| Trial TorchTrainer_a1400_00004 result                         |
+---------------------------------------------------------------+
| checkpoint_dir_name                         checkpoint_000000 |
| time_this_iter_s                                    109.00794 |
| time_total_s                                        109.00794 |
| training_iteration                                          1 |
| accuracy                                              0.15445 |
| loss                                                 18.97903 |
| summary/epoch/0                                           1.0 |
| summary/train_acc/0                       0.15378610460577674 |
| summary/train_loss/0                        2.832206237606886 |
| summary/val_acc/0                          0.1544461778471139 |
| summary/val_loss/0       

2024-04-14 09:01:49,143	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2024-04-14 09:01:49,186	INFO tune_controller.py:445 -- Restoring the run from the latest experiment state file: experiment_state-2024-04-14_07-25-24.json


Outer fold 4, inner fold 2 - number of samples: 1282
Tuning hyperparameters for wide_resnet50_hpt_outer_4_inner_2...
Defaulting to ASHA scheduler (no scheduler provided or not an instance of TrialScheduler)
Restoring tuner from path /root/ray_results/wide_resnet50_hpt_outer_4_inner_2


2024-04-14 09:01:49,509	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/wide_resnet50_hpt_outer_4_inner_2' in 0.1955s.


+----------------------------------------------------------------------+
| Configuration for experiment     wide_resnet50_hpt_outer_4_inner_2   |
+----------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator               |
| Scheduler                        AsyncHyperBandScheduler             |
| Number of trials                 5                                   |
+----------------------------------------------------------------------+

View detailed results here: /root/ray_results/wide_resnet50_hpt_outer_4_inner_2
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-14_07-52-09_892030_20145/artifacts/2024-04-14_07-25-24/wide_resnet50_hpt_outer_4_inner_2/driver_artifacts`

Trial status: 5 TERMINATED
Current time: 2024-04-14 09:01:49. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:V100)
+-------------------------------------------

## Examine results


In [8]:
import json
import re
import pandas as pd

from pprint import pprint

In [9]:
best_experiments = []
for i in range(5):
  for j in range(3):
    file_summary = f"hpt_wide_resnet50_hpt_outer_{i}_inner_{j}_exp_summary.csv"
    file_path = os.path.join(persistence_path, file_summary)
    tmp_df = pd.read_csv(file_path, usecols=["loss", "accuracy", "trial_id", "time_total_s"])
    tmp_df = tmp_df.loc[tmp_df['accuracy'] == tmp_df['accuracy'].max()]
    tmp_df['outer_fold'] = i
    tmp_df['inner_fold'] = j
    best_experiments.append(
        tmp_df
    )

best_experiments = pd.concat(best_experiments)
best_experiments = best_experiments.sort_values(by=['accuracy', 'loss', 'time_total_s'], ascending=[False, True, True])
best_experiments

,loss,accuracy,trial_id,time_total_s,outer_fold,inner_fold
2,1.901282,0.371295,8d8f0_00002,1690.489108,2,0
1,2.407765,0.344774,d2735_00001,835.887733,2,1
0,2.556820,0.341654,9d196_00000,1743.971443,1,0
0,2.075537,0.305772,998b8_00000,433.088491,3,0
0,2.112708,0.304688,0c78b_00000,1535.406898,0,2
0,2.022516,0.299532,a1400_00000,1718.978152,4,1
0,3.121286,0.294852,e8bbe_00000,821.023257,4,0
0,2.097283,0.290625,cf44b_00000,1547.026404,0,1
0,2.013552,0.276131,116e1_00000,1656.248230,0,0
0,2.211824,0.256250,c540c_00000,1668.311860,2,2


In [ ]:
#best_experiments.to_csv(os.path.join(persistence_path, f"HP_RANKING.{selected_model}_overview.csv"), index=False)

In [10]:
best_params_path = os.path.join(persistence_path,
                                f"wide_resnet50_hpt_outer_{best_experiments.iloc[0].outer_fold}_inner_{best_experiments.iloc[0].inner_fold}",
                                "params.json")
with open(best_params_path, 'r') as params_file:
  best_params = json.load(params_file)

best_params = {k: v for k, v in best_params['train_loop_config'].items() if k in search_space.keys()}

pprint(best_params)

{'batch_size': 32, 'epochs': 5, 'lr': 0.00010777744580116128}


# Cross-validation on entire dataset


In [11]:
ciri_trainer = CIRI_trainer(model=selected_model,
                            data_folders=data_folders,
                            data_prop=0.8)

In [12]:
persistence_path = os.path.join(project_folder, 'checkpoints', f'CV_{selected_model}')
os.makedirs(persistence_path, exist_ok=True)

In [13]:
cv_whole = ciri_trainer.cross_validate(
    run_name="wide_resnet50_cv",
    config={
        **best_params,
        "additional_metrics": ['precision', 'recall', 'f1', 'confusion_matrix']
    },
    outer_cv_k=5,
    inner_cv_k=0,
    tune_hyperparams=False,
    results_persist_dir=persistence_path,
    start_fold=(4,0)
)

Outer fold 4 - number of samples: 9606


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-04-14 19:32:14,130	INFO worker.py:1752 -- Started a local Ray instance.
2024-04-14 19:32:15,849	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `<FrameworkTrainer>(...)`.
2024-04-14 19:32:15,861	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949



View detailed results here: /root/ray_results/wide_resnet50_cv_outer_4
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-14_19-32-11_529013_4082/artifacts/2024-04-14_19-32-15/wide_resnet50_cv_outer_4/driver_artifacts`


(TrainTrainable pid=5548) 2024-04-14 19:32:23.503976: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=5548) 2024-04-14 19:32:23.504028: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=5548) 2024-04-14 19:32:23.505430: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainTrainable pid=5548) 2024-04-14 19:32:24.822003: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Training started with configuration:
+---------------------------------------------------------------+
| Training config                                               |
+---------------------------------------------------------------+
| train_loop_config/additional_metrics     ...onfusion_matrix'] |
| train_loop_config/batch_size                               32 |
| train_loop_config/data_folders           ...ugmented_images'] |
| train_loop_config/data_prop                               0.8 |
| train_loop_config/epochs                                    5 |
| train_loop_config/lr                   0.00010777744580116128 |
| train_loop_config/model                       wide_resnet50_2 |
| train_loop_config/sample_indices                              |
| train_loop_config/train_test_idx         ...1, 12002, 12003]) |
+---------------------------------------------------------------+


...


Training finished iteration 4 at 2024-04-14 21:33:20. Total running time: 2hr 1min 4s
+---------------------------------------------+
| Training result                             |
+---------------------------------------------+
| checkpoint_dir_name       checkpoint_000003 |
| time_this_iter_s                  488.99802 |
| time_total_s                      7254.7423 |
| training_iteration                        4 |
| accuracy                            0.45458 |
| confusion_matrix       ...11, 8, 7, 7, 11]) |
| f1                                  0.43183 |
| loss                                1.60006 |
| precision                           0.51559 |
| recall                              0.45458 |
| summary/epoch/0                         1.0 |
| summary/epoch/1                         2.0 |
| summary/epoch/2                         3.0 |
| summary/epoch/3                         4.0 |
| summary/train_acc/0     0.28052083333333333 |
| summary/train_acc/1     0.40020833333333333 |
|

Epoch (training) 5:  45%|████▍     | 135/301 [02:46<03:15,  1.18s/it]
(RayTrainWorker pid=5624) /usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
(RayTrainWorker pid=5624)   warnings.warn(
Epoch (test) 5: 100%|██████████| 76/76 [01:36<00:00,  1.27s/it]
(RayTrainWorker pid=5624) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/wide_resnet50_cv_outer_4/TorchTrainer_b33ac_00000_0_2024-04-14_19-32-16/checkpoint_000004)



Training finished iteration 5 at 2024-04-14 21:41:30. Total running time: 2hr 9min 13s
+---------------------------------------------+
| Training result                             |
+---------------------------------------------+
| checkpoint_dir_name       checkpoint_000004 |
| time_this_iter_s                  489.09886 |
| time_total_s                     7743.84116 |
| training_iteration                        5 |
| accuracy                            0.51208 |
| confusion_matrix       ...0, 8, 11, 8, 10]) |
| f1                                  0.50517 |
| loss                                1.45386 |
| precision                           0.55147 |
| recall                              0.51208 |
| summary/epoch/0                         1.0 |
| summary/epoch/1                         2.0 |
| summary/epoch/2                         3.0 |
| summary/epoch/3                         4.0 |
| summary/epoch/4                         5.0 |
| summary/train_acc/0     0.28052083333333333 |


2024-04-14 21:41:31,685	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-04-14 21:41:31,699	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/wide_resnet50_cv_outer_4' in 0.0488s.



Training completed after 5 iterations at 2024-04-14 21:41:31. Total running time: 2hr 9min 15s



# Transfer learning

In [ ]:
ciri_trainer = CIRI_trainer(model=selected_model,
                            data_folders=data_folders,
                            data_prop=0.8)

In [ ]:
project_folder = ... # Redefine here project folder if necessary
persistence_path = os.path.join(project_folder, 'checkpoints', f'TransferLearning_{selected_model}')
os.makedirs(persistence_path, exist_ok=True)

In [ ]:
tl_results = ciri_trainer.train(
    run_name=f'{selected_model}_transfer_learning',
    config={
        **best_params,
        'additional_metrics': ['precision', 'recall', 'f1', 'confusion_matrix'],
        'weights': 'IMAGENET1K_V2',
    },
    persist_dir=persistence_path
)